In [46]:
import pandas as pd
import seaborn as sns
import numpy as np

## Load the dataset


In [ ]:
train_data=pd.read_csv('datasets/credit_scoring_train.csv')
test_data=pd.read_csv('datasets/credit_scoring_test.csv')
sample_data=pd.read_csv('datasets/credit_scoring_sample.csv')

In [ ]:
sample_data

## Preprocess the data

In [ ]:
sample_data.fillna(value='mean')

In [ ]:
X=sample_data.drop('SeriousDlqin2yrs',axis=1)
y=sample_data['SeriousDlqin2yrs']

In [ ]:
for col in X.columns:
    X[col] = X[col].fillna(X[col].median())

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [ ]:
lr = LogisticRegression(random_state=5, class_weight="balanced")

In [ ]:
parameters = {"C": (0.0001, 0.001, 0.01, 0.1, 1, 10)}

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

Question 3. Perform a Grid Search with the scoring metric “roc_auc” for the parameter C. Which value of the parameter C is optimal?

In [ ]:
grid_search = GridSearchCV(lr, parameters, n_jobs=-1, scoring="roc_auc", cv=skf)
grid_search = grid_search.fit(X, y)
grid_search.best_estimator_

Question 4. Can we consider the best model stable? The model is stable if the standard deviation on validation is less than 0.5%. Save the ROC AUC value of the best model, it will be useful for the following tasks.

In [ ]:
grid_search.cv_results_["std_test_score"][1]

In [ ]:
grid_search.best_score_

## Feature importance

Question 9. What feature has the weakest impact in Random Forest model?

age

NumberOfTime30-59DaysPastDueNotWorse

DebtRatio

NumberOfTimes90DaysLate

NumberOfTime60-89DaysPastDueNotWorse

MonthlyIncome

NumberOfDependents

In [ ]:
from sklearn.preprocessing import StandardScaler

lr = LogisticRegression(C=0.001, random_state=5, class_weight="balanced")
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

pd.DataFrame(
    {"feat": X.columns, "coef": lr.coef_.flatten().tolist()}
).sort_values(by="coef", ascending=False)

,feat,coef
1,NumberOfTime30-59DaysPastDueNotWorse,0.723421
3,NumberOfTimes90DaysLate,0.516656
4,NumberOfTime60-89DaysPastDueNotWorse,0.195486
6,NumberOfDependents,0.101722
2,DebtRatio,-0.024068
5,MonthlyIncome,-0.163015
0,age,-0.417115


Exception ignored in: <function ResourceTracker.__del__ at 0x7fdc24871580>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7e7ca2c7d580>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x79e5f1475580>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/reso

Question 6. Calculate how much DebtRatio affects our prediction using the softmax function. What is its value?



In [47]:
print((np.exp(lr.coef_[0]) / np.sum(np.exp(lr.coef_[0])))[2])


0.11423641577786513


Question 7. Let’s see how we can interpret the impact of our features. For this, recalculate the logistic regression with absolute values, that is without scaling. Next, modify the customer’s age by adding 20 years, keeping the other features unchanged. How many times will the chance that the customer will not repay their debt increase? You can find an example of the theoretical calculation here.

In [51]:
lr = LogisticRegression(C=0.001, random_state=5, class_weight="balanced")
lr.fit(X, y)

pd.DataFrame(
    {"feat": X.columns, "coef": lr.coef_.flatten().tolist()}
).sort_values(by="coef", ascending=False)

/home/yogi/workspace/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,feat,coef
1,NumberOfTime30-59DaysPastDueNotWorse,0.544094
3,NumberOfTimes90DaysLate,0.456189
6,NumberOfDependents,0.113446
2,DebtRatio,-0.000011
5,MonthlyIncome,-0.000012
0,age,-0.017585
4,NumberOfTime60-89DaysPastDueNotWorse,-0.035489


In [52]:
np.exp(lr.coef_[0][0] * 20)


np.float64(0.7034894518245197)

## Random Forest

In [53]:
from sklearn.ensemble import RandomForestClassifier


In [54]:
randf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=-1,class_weight='balanced')

In [55]:
parameters = {
    "max_features": [1, 2, 4],
    "min_samples_leaf": [3, 5, 7, 9],
    "max_depth": [5, 10, 15],
}

In [60]:
rand_grid=GridSearchCV(randf,parameters,cv=skf,scoring="roc_auc",verbose=True)

How much higher is the ROC AUC of the best random forest model than that of the best logistic regression on validation? Select the closest answer

In [61]:
rand_grid.fit(X,y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=5, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              n_jobs=-1, random_state=42),
             param_grid={'max_depth': [5, 10, 15], 'max_features': [1, 2, 4],
                         'min_samples_leaf': [3, 5, 7, 9]},
             scoring='roc_auc', verbose=True)

In [64]:
rand_grid.best_score_ - grid_search.best_score_

np.float64(0.04103592227190078)

Question 9. What feature has the weakest impact in Random Forest model?



In [69]:
X.columns[
    np.argmin(rand_grid.best_estimator_.feature_importances_)
]

'NumberOfDependents'

## Bagging

In [73]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

parameters = {
    "max_features": [2, 3, 4],
    "max_samples": [0.5, 0.7, 0.9],
    "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100],
}

In [74]:
Question 11. Fit a bagging classifier with random_state=42. For the base classifiers, use 100 logistic regressors and use RandomizedSearchCV instead of GridSearchCV. It will take a lot of time to iterate over all 54 variants, so set the maximum number of iterations for RandomizedSearchCV to 20. Don’t forget to set the parameters cv and random_state=1. What is the best ROC AUC you achieve?

Object `achieve` not found.
